# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Collect each filepath
for root, dirs, files in os.walk(filepath):
    if dirs == []:
        continue
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
# Store all data in the full_data_rows_list
for f in file_path_list:
 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
             
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))


# creating a csv file containing all necessary data
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
query = "CREATE TABLE IF NOT EXISTS song_info "
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

#### A description of the `SELECT` query below
- **SELECT artist, song, length**: The song's information we are interested in is `artist`, `song`, and `length`.
- **FROM song_info**: We can retrieve the above information from the `song_info` table.
- **WHERE sessionId=338 AND itemInSession=4**: To determine a song that was heard by a certain user, it is requried to know the values of both `sessionId` and `itemInSession`. This is the reason why those two columns constitute the composite key and `WHERE` clause in the `SELECT` statement.

In [10]:
query = "SELECT artist, song, length FROM song_info WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
query = "CREATE TABLE IF NOT EXISTS users_selection "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
# Insert data
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_selection (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
                        

#### A description of the `SELECT` query below
- **SELECT itemInSession, artist, song, firstName, lastName**: The `itemInSession` is placed at the front to show that the retrieved results are sorted by `itemInSession`. The other columns are information we are interested in.
- **FROM users_selection**: The `users_selection` table contains information we are interested in.
- **WHERE userId=10 AND sessionId=182**: To determine songs that was heard by a certain user, it is requried to know who listened to a music(`userId`) and when he/she did(`sessionId`). Thus, those two columns constitute the composite key. Also, to make sure songs are sorted by `itemInSession`, it is necessary to place `itemInSession` in the primary keys as a clustering column.

In [13]:
## Verify
query = "SELECT itemInSession, artist, song, firstName, lastName FROM users_selection WHERE userId=10 AND sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.iteminsession, row.artist, row.song, row.firstname, row.lastname)    

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [14]:
query = "CREATE TABLE IF NOT EXISTS users_by_song "
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [15]:
# Insert data
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_song (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
                        

#### A description of the `SELECT` query below
- **SELECT firstName, lastName**: The two columns are what we are looking for.
- **FROM users_by_song**: The `users_by_song` table contains information we are interested in.
- **WHERE song='All Hands Against His Own'**: To retrieve all users who heard the song titled 'All Hands Against His Own', we need not only `song`, but we also need `userId`. If we miss `userId` in the primary key, we would end up retrieving only one user rather than every users that heard the song before. In sum, constructing the composite key with `song` and `userId` allow `WHERE` condition to produce multiple results.

In [16]:
## Verify
query = "SELECT firstName, lastName FROM users_by_song WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
query = "DROP TABLE song_info"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [18]:
query = "DROP TABLE users_selection"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [19]:
query = "DROP TABLE users_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()